In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
from standard_functions import getAPI

Do not ever put API's in your repositories. This one regenerates after 24 hours and doesn't have any sensitive information behind it. But do as I say, not as I do.

In [2]:
myApi = getAPI()
lol_watcher = LolWatcher(myApi)

region = 'na1'


In [3]:
def get_puuid(name):
    """
    Input is summoner name. Outputs players puuid.
    """
    return lol_watcher.summoner.by_name(region, name)['puuid']
def get_matchlist(name):
    """
    Input is name. Output is list of matches.
    """
    return lol_watcher.match.matchlist_by_puuid(region, get_puuid(name), count=100, type='ranked')


In [5]:
summoners = ['Zealfire', 'SmurfHD', 'DumbAmerican', 'About 10 Inches', 'Exprience', 'Leaders1', 
             'Kehlee', 'KybitOW', 'DodoMuncher', 'Mango Bleach']

In [6]:
get_matchlist('Zealfire')

['NA1_4348171334',
 'NA1_4289259129',
 'NA1_4287742329',
 'NA1_4287694201',
 'NA1_4287648113',
 'NA1_4286956599',
 'NA1_4286785334',
 'NA1_4286667294',
 'NA1_4285375587',
 'NA1_4285360882',
 'NA1_4285334783',
 'NA1_4285247595',
 'NA1_4284641450',
 'NA1_4284573748',
 'NA1_4282617016',
 'NA1_4282651208',
 'NA1_4282568419',
 'NA1_4282560428',
 'NA1_4279275380',
 'NA1_4279290874',
 'NA1_4272162489',
 'NA1_4270474097',
 'NA1_4270373912',
 'NA1_4270325535',
 'NA1_4270310445',
 'NA1_4269678873',
 'NA1_4269623819',
 'NA1_4265939472',
 'NA1_4265052963',
 'NA1_4265007120',
 'NA1_4265011466',
 'NA1_4264596019',
 'NA1_4264558755',
 'NA1_4263428731',
 'NA1_4262510186',
 'NA1_4262424973',
 'NA1_4259122335',
 'NA1_4259076750',
 'NA1_4258616238',
 'NA1_4257716745',
 'NA1_4251471488',
 'NA1_4248601449',
 'NA1_4248509505',
 'NA1_4247935338',
 'NA1_4243646495',
 'NA1_4242625676',
 'NA1_4242377491',
 'NA1_4242051531',
 'NA1_4241728549',
 'NA1_4241732889',
 'NA1_4240856758',
 'NA1_4240829863',
 'NA1_423986

In [8]:
pd.DataFrame([lol_watcher.match.by_id(region, 'NA1_4205025589')['info']])

,gameCreation,gameDuration,gameEndTimestamp,gameId,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,mapId,participants,platformId,queueId,teams,tournamentCode
0,1655686497000,1449,1655687977149,4348171334,CLASSIC,teambuilder-match-4348171334,1655686527674,MATCHED_GAME,12.11.446.9344,11,"[{'assists': 6, 'baronKills': 0, 'bountyLevel'...",NA1,440,"[{'bans': [{'championId': 54, 'pickTurn': 1}, ...",


In [9]:
pd.DataFrame([lol_watcher.match.by_id(region, 'NA1_NA1_4205025589')['info']['participants'][0]])

,assists,baronKills,bountyLevel,challenges,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,...,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,6,0,6,"{'12AssistStreakCount': 0, 'abilityUses': 93, ...",13245,15,133,Quinn,0,5,...,1062,1,1,3,0,18,1,0,7,True


In [10]:
data = lol_watcher.league.entries(region, queue='RANKED_SOLO_5x5', tier='GOLD', division='II', page=500)
len(data)

0

In [25]:
lol_watcher.match.by_id(region, 'NA1_4205025589')['info'].keys()

dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])

In [13]:
lol_watcher.match.by_id(region, 'NA1_4205025589')['info']

{'gameCreation': 1643938709000,
 'gameDuration': 1555,
 'gameEndTimestamp': 1643940333289,
 'gameId': 4205025589,
 'gameMode': 'CLASSIC',
 'gameName': 'teambuilder-match-4205025589',
 'gameStartTimestamp': 1643938777865,
 'gameType': 'MATCHED_GAME',
 'gameVersion': '12.3.421.5967',
 'mapId': 11,
 'participants': [{'assists': 3,
   'baronKills': 0,
   'bountyLevel': 0,
   'challenges': {'12AssistStreakCount': 0,
    'abilityUses': 278,
    'acesBefore15Minutes': 0,
    'alliedJungleMonsterKills': 1.5,
    'baronTakedowns': 0,
    'blastConeOppositeOpponentCount': 0,
    'bountyGold': 0,
    'buffsStolen': 0,
    'controlWardsPlaced': 0,
    'damagePerMinute': 3326.0170831635774,
    'damageTakenOnTeamPercentage': 0.1880711600609258,
    'dancedWithRiftHerald': 0,
    'deathsByEnemyChamps': 10,
    'doubleAces': 0,
    'dragonTakedowns': 0,
    'effectiveHealAndShielding': 0,
    'elderDragonKillsWithOpposingSoul': 0,
    'elderDragonMultikills': 0,
    'enemyChampionImmobilizations': 28

In [31]:
lol_watcher.match.by_id(region, 'NA1_4205025589')['info']['teams'][0]['objectives'].keys()

dict_keys(['baron', 'champion', 'dragon', 'inhibitor', 'riftHerald', 'tower'])

In [38]:
pd.DataFrame(lol_watcher.match.by_id(region, 'NA1_4205025589')['info']['teams'][0]['objectives']).rename(columns=['first_baron', 'first_champion', 'first_dragon', 'first_inhibitor', 'first_riftHerald', 'first_tower'])

TypeError: 'list' object is not callable

In [24]:
pd.DataFrame(lol_watcher.match.timeline_by_match(region, 'NA1_4205025589')['info']['frames'])

,events,participantFrames,timestamp
0,"[{'realTimestamp': 1643938777750, 'timestamp':...","{'1': {'championStats': {'abilityHaste': 0, 'a...",0
1,"[{'levelUpType': 'NORMAL', 'participantId': 8,...","{'1': {'championStats': {'abilityHaste': 0, 'a...",60022
2,"[{'levelUpType': 'NORMAL', 'participantId': 2,...","{'1': {'championStats': {'abilityHaste': 0, 'a...",120034
3,"[{'creatorId': 4, 'timestamp': 120298, 'type':...","{'1': {'championStats': {'abilityHaste': 0, 'a...",180072
4,"[{'itemId': 2055, 'participantId': 6, 'timesta...","{'1': {'championStats': {'abilityHaste': 0, 'a...",240095
5,"[{'itemId': 1042, 'participantId': 6, 'timesta...","{'1': {'championStats': {'abilityHaste': 0, 'a...",300124
6,"[{'level': 4, 'participantId': 9, 'timestamp':...","{'1': {'championStats': {'abilityHaste': 0, 'a...",360145
7,"[{'killerId': 0, 'laneType': 'TOP_LANE', 'posi...","{'1': {'championStats': {'abilityHaste': 0, 'a...",420162
8,"[{'levelUpType': 'NORMAL', 'participantId': 9,...","{'1': {'championStats': {'abilityHaste': 0, 'a...",480189
9,"[{'afterId': 0, 'beforeId': 3802, 'goldGain': ...","{'1': {'championStats': {'abilityHaste': 0, 'a...",540196


In [ ]:
pd.DataFrame.rename()